# Grid2x2 using RLlib

In [1]:
import csv
import os

import ray
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility
from ray.tune.registry import register_env

from envs import MultiAgentSumoEnv
from observation import Grid2x2ObservationFunction
from reward_functions import combined_reward

In [2]:
import random
import numpy as np
import torch

TEST_NUM = 3
SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ENV_NAME = "grid2x2"

In [3]:
def train_env_creator(args):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(**env_params)
    return env

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

train_env = MultiAgentEnvCompatibility(train_env_creator({}))

config: PPOConfig
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .framework(framework="torch")
    .rollouts(
        rollout_fragment_length=100,
        num_rollout_workers=10,
    )
    .training(
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        train_batch_size=1000,
        sgd_minibatch_size=100,
        num_sgd_iter=10,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_num_workers=1,
        evaluation_sample_timeout_s=300,
    )
    .debugging(seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "1")))
    .multi_agent(
        policies=set(train_env.env.ts_ids),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
)

delta_wait_time_reward 0.875


In [5]:
csv_dir = os.path.join("outputs",ENV_NAME,f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

## Play Untrained Agent

In [6]:
def eval_env_creator(csv_path, tb_log_dir):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(eval=True, csv_path=csv_path, tb_log_dir=tb_log_dir, **env_params)
    return env

In [7]:
ray.init()

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_untrained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

2023-06-01 23:29:54,657	INFO worker.py:1625 -- Started a local Ray instance.


In [8]:
algo = config.build()

2023-06-01 23:29:58,627	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-01 23:29:58,659	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=12372) delta_wait_time_reward 0.875
(RolloutWorker pid=24364) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=24304) 2023-06-01 23:30:06,230	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-01 23:30:08,879	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=22352) delta_wait_time_reward 0.875 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=18708) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                      [repeated 9x across cluster]


(RolloutWorker pid=22352) 2023-06-01 23:30:13,005	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-01 23:30:13,094	INFO trainable.py:172 -- Trainable.setup took 14.440 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [9]:
algo.evaluate()

Step #3600.00 (1ms ~= 1000.00*RT, ~231000.00UPS, TraCI: 249ms, vehicles TOT 1715 ACT 231 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~231000.00UPS, TraCI: 249ms, vehicles TOT 1715 ACT 231 BOT 3 ACT 3 BUF 0)                      


{'evaluation': {'episode_reward_max': -320852.8744288877,
  'episode_reward_min': -320852.8744288877,
  'episode_reward_mean': -320852.8744288877,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -76832.12950596321,
   '2': -83696.77696430053,
   '5': -79422.03550824955,
   '6': -80901.93245037406},
  'policy_reward_max': {'1': -76832.12950596321,
   '2': -83696.77696430053,
   '5': -79422.03550824955,
   '6': -80901.93245037406},
  'policy_reward_mean': {'1': -76832.12950596321,
   '2': -83696.77696430053,
   '5': -79422.03550824955,
   '6': -80901.93245037406},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-320852.8744288877],
   'episode_lengths': [720],
   'policy_1_reward': [-76832.12950596321],
   'policy_2_reward': [-83696.77696430053],
   'policy_5_reward': [-79422.03550824955],
   'policy_6_reward': [-80901.93245037406]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 4.287893993679264,
   'mean_inferen

In [10]:
ray.shutdown()

## Train RL Agent

In [11]:
ray.init()

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(train_env_creator(config)))

2023-06-01 23:34:30,062	INFO worker.py:1625 -- Started a local Ray instance.


In [12]:
algo = config.build()

2023-06-01 23:34:33,614	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=3572) delta_wait_time_reward 0.875
(RolloutWorker pid=3572) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=4300) 2023-06-01 23:34:40,269	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-01 23:34:40,640	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-01 23:34:44,696	INFO trainable.py:172 -- Trainable.setup took 11.076 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [13]:
TRAIN_EPS = 1400  # 720 * 1400 == 1_008_000 total timesteps
CHECKPOINT_FREQ = 200
assert TRAIN_EPS % CHECKPOINT_FREQ == 0

for i in range(TRAIN_EPS):
    results = algo.train()

    if (i+1) % CHECKPOINT_FREQ == 0:
        algo.save(os.path.join("ray_checkpoints","grid2x2",f"test_{TEST_NUM}"))

2023-06-01 23:34:58,880	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Step #3600.00 (1ms ~= 1000.00*RT, ~96000.00UPS, TraCI: 95ms, vehicles TOT 2094 ACT 96 BUF 49ms, vehicles TOT 3 ACT 3 BUF 0)      
(RolloutWorker pid=1724) delta_wait_time_reward 0.875 [repeated 10x across cluster]
(RolloutWorker pid=1724) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]
Step #3600.00 (1ms ~= 1000.00*RT, ~127000.00UPS, TraCI: 103ms, vehicles TOT 2047 ACT 127 B49ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~140000.00UPS, TraCI: 101ms, vehicles TOT 2099 ACT 140 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 106ms, vehicles TOT 2055 ACT 168 BUF 51)            0ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~192000.00UPS, TraCI: 111ms, vehicles TOT 1703 ACT 192 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (2ms ~= 500.00*RT, ~100000.00UPS, TraCI: 109ms, vehicles TOT 1710 ACT 200 BU49ms, vehicles TOT 3 ACT 3 BUF 0

(RolloutWorker pid=24796) Error: tcpip::Socket::accept() Unable to create listening socket: Address already in use
(RolloutWorker pid=24796) Quitting (on error).
(RolloutWorker pid=1724) Quitting (on error).
(RolloutWorker pid=24796) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 50380 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=24796)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles TOT 2106 ACT 44 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 75ms, vehicles TOT 2106 ACT 43 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~40000.00UPS, TraCI: 72ms, vehicles TOT 2106 ACT 40 BUF 43ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~44000.00UPS, TraCI: 67ms, vehicles TOT 2106 ACT 44 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~46000.00UPS, TraCI: 72ms, vehicles TOT 2106 ACT 46 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 76ms, vehicles TOT 2106 ACT 47 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 79ms, vehicles TOT 2106 ACT 45 BUF 0)               9ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~53000.00UPS, TraCI: 74ms, vehicles TOT 2106 ACT 53 BUF OT

(RolloutWorker pid=13588) Quitting (on error).
(RolloutWorker pid=13588) Quitting (on error).
(RolloutWorker pid=13588) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 51255 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=13588)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


...

Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 77ms, vehicles TOT 2106 ACT 42 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~42000.00UPS, TraCI: 78ms, vehicles TOT 2106 ACT 42 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~41000.00UPS, TraCI: 78ms, vehicles TOT 2106 ACT 41 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 79ms, vehicles TOT 2106 ACT 49 BUF 0)               27ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 78ms, vehicles TOT 2106 ACT 46 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~45000.00UPS, TraCI: 63ms, vehicles TOT 2106 ACT 45 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 66ms, vehicles TOT 2106 ACT 45 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles TOT 2106 ACT 47 BUF 0)               4

(RolloutWorker pid=19416) Quitting (on error).
(RolloutWorker pid=19416) Quitting (on error).
(RolloutWorker pid=19416) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 52330 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=19416)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 73ms, vehicles TOT 2106 ACT 44 BUF 0)               8ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~42000.00UPS, TraCI: 70ms, vehicles TOT 2106 ACT 42 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 66ms, vehicles TOT 2106 ACT 44 BUF 0)               30ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~41000.00UPS, TraCI: 73ms, vehicles TOT 2106 ACT 41 BUF 32ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~39000.00UPS, TraCI: 72ms, vehicles TOT 2106 ACT 39 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~40000.00UPS, TraCI: 66ms, vehicles TOT 2106 ACT 40 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~40000.00UPS, TraCI: 74ms, vehicles TOT 2106 ACT 40 BUF 28ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~43000.00UPS, TraCI: 56ms, vehicles TOT 2106 ACT 43 BUF 30

(RolloutWorker pid=21580) Error: tcpip::Socket::accept() Unable to create listening socket: Address already in use
(RolloutWorker pid=21580) Quitting (on error).
(RolloutWorker pid=21580) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 52472 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=21580)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles TOT 2106 ACT 41 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles TOT 2106 ACT 40 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~46000.00UPS, TraCI: 74ms, vehicles TOT 2106 ACT 46 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 76ms, vehicles TOT 2106 ACT 47 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~45000.00UPS, TraCI: 73ms, vehicles TOT 2106 ACT 45 BUF 36ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~48000.00UPS, TraCI: 66ms, vehicles TOT 2106 ACT 48 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 55ms, vehicles TOT 2106 ACT 38 BUF 0)               44ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 70ms, vehicles TOT 2106 ACT 43 BUF 0)               26

In [15]:
ray.shutdown()

## Play Trained Agent

In [28]:
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_trained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

In [29]:
ray.init()

2023-06-02 11:43:03,265	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.4.0


In [30]:
from ray.rllib.algorithms.ppo import PPO

checkpoint_path = os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}",f"checkpoint_{TRAIN_EPS:06}")
checkpoint_path = os.path.abspath(checkpoint_path)

ppo_agent = PPO.from_checkpoint(checkpoint_path)

2023-06-02 11:43:09,349	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=27516) delta_wait_time_reward 0.875
(RolloutWorker pid=27516) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 25ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=20868) 2023-06-02 11:43:18,959	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-02 11:43:19,459	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=24788) delta_wait_time_reward 0.875 [repeated 10x across cluster]
(RolloutWorker pid=21644) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 15ms, vehicles TOT 0 ACT 0 BUF 0)                      [repeated 9x across cluster]


(RolloutWorker pid=24788) 2023-06-02 11:43:27,470	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-02 11:43:27,655	INFO trainable.py:172 -- Trainable.setup took 18.290 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [31]:
ppo_agent.evaluate()

Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 220ms, vehicles TOT 2106 ACT 48 BUF 0)              83ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 220ms, vehicles TOT 2106 ACT 48 BUF 0)              83ms, vehicles TOT 3 ACT 3 BUF 0)      


{'evaluation': {'episode_reward_max': -79401.3654629666,
  'episode_reward_min': -79401.3654629666,
  'episode_reward_mean': -79401.3654629666,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -19981.939879322545,
   '2': -20625.42274046845,
   '5': -19120.736150769924,
   '6': -19673.26669240556},
  'policy_reward_max': {'1': -19981.939879322545,
   '2': -20625.42274046845,
   '5': -19120.736150769924,
   '6': -19673.26669240556},
  'policy_reward_mean': {'1': -19981.939879322545,
   '2': -20625.42274046845,
   '5': -19120.736150769924,
   '6': -19673.26669240556},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-79401.3654629666],
   'episode_lengths': [720],
   'policy_1_reward': [-19981.939879322545],
   'policy_2_reward': [-20625.42274046845],
   'policy_5_reward': [-19120.736150769924],
   'policy_6_reward': [-19673.26669240556]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 4.45221102022761,
   'mean_infe

In [27]:
ray.shutdown()

(RolloutWorker pid=26832) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 42ms, vehicles TOT 0 ACT 0 BUF 0)                     
